## 🪄 Install `wandb` library and login


Start by installing the library and logging in to your free account.



In [1]:
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=9b4cc00015aee2ec8ce1072c891b7fc2a5725919c187c68ffad9498167d3f1a3
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [2]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
#@title Imports
import torch
import torch.optim as optim

import pdb
import time
import torch.nn as nn
import torch.nn.functional as F
from keras.datasets import mnist
from torchvision import datasets, transforms
import torch.nn.init as init
import numpy as np
import matplotlib.pyplot as plt


In [4]:
#@title Initialize AutoEncoder Parameters

latent_dim  = 10 # dimensional latent space
latent_dimsQ2 = [4,10,40]
latent_dimsQ3 = [5,10,20,40]

batch_size = 64
epochs = 20
lr = 0.001
kernel_size = (3,3)
weight_decay = 0.01
betas = (0.9, 0.999)

In [5]:
#@title Load the MNIST dataset


# transformer that  converters the data to PyTorch tensors and normalize the data
transform = transforms.Compose([
    transforms.ToTensor()])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Download and load the test data
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

# Q3 
trainDecorrelation = torch.utils.data.DataLoader(trainset, batch_size=4000, shuffle=True)

# Q4
trainloaderMLP = torch.utils.data.DataLoader(trainset, batch_size=20, shuffle=True)

# Create list with 4 batches in size 20
mlpTrainSet = []
num_batches = 4

for i in range(num_batches):
    mlpTrainSet.append(next(iter(trainloaderMLP))) 


100%|██████████| 9912422/9912422 [00:00<00:00, 126516455.97it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 37997394.55it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31225241.54it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5822288.74it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



In [12]:
#@title Define AE

# TO use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# stride=2:
# output feature maps will have half the height and width of the input feature maps.
# padding='same'
# ensures that the output feature maps have the same height and width as the input feature maps.
# Using these options together in a convolutional layer in a neural network helps to reduce the size of the feature maps
# no spatial structure - autoencoder should only capture the essential features of the image, while disregarding any spatial relationships between pixels.
################ original ################
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()

        # init - layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.fc1 = nn.Linear(32 * 7 * 7, latent_dim)
  
        # inits - weights (Xavier initialization)
        for layer in self.modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
        
        # save total parameters of the encoder
        self.total_params = sum(p.numel() for p in self.parameters())


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        
        # init - layers
        self.fc1 = nn.Linear(latent_dim, 32 * 7 * 7)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.conv1 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=16)
        self.conv2 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1)
        
        # inits - weights (Xavier initialization)
        for layer in self.modules():
            if isinstance(layer, nn.ConvTranspose2d) or isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
        
        # save total parameters of the decoder        
        self.total_params = sum(p.numel() for p in self.parameters())


    def forward(self, x):
        x = self.fc1(x)
        x = x.view(x.size(0), 32, 7, 7) 
        x = F.relu(self.conv1(self.bn1(x)))
        x = F.sigmoid(self.conv2(self.bn2(x)))
        return x


################ II - more layers ################

# class Encoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()

#         # init - layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1)
#         self.bn1 = nn.BatchNorm2d(num_features=16)
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
#         self.bn2 = nn.BatchNorm2d(num_features=32)
#         self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
#         self.bn3 = nn.BatchNorm2d(num_features=64)
        
#         self.fc1 = nn.Linear(64 * 4 * 4, latent_dim)
  
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the encoder
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = self.fc1(x)
#         return x

# class Decoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()
        
#         # init - layers
#         self.fc1 = nn.Linear(latent_dim, 64 * 4 * 4)
#         self.bn1 = nn.BatchNorm2d(num_features=64)
#         self.conv1 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1)
#         self.bn2 = nn.BatchNorm2d(num_features=32)
#         self.conv2 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1)
#         self.bn3 = nn.BatchNorm2d(num_features=16)
#         self.conv3 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1)
       
        
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.ConvTranspose2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the decoder        
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):
#         x = self.fc1(x)
#         x = x.view(x.size(0), 64,4, 4) 

#         x = F.relu(self.conv1(self.bn1(x)))

#         x = F.relu(self.conv2(self.bn2(x)))

#         x = F.sigmoid(self.conv3(self.bn3(x)))

#         return x

############### II - less layers ################
# class Encoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()

#         # init - layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1)
#         self.bn1 = nn.BatchNorm2d(num_features=16)
 
#         self.fc1 = nn.Linear(16 * 14 * 14, latent_dim)
  
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the encoder
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = self.fc1(x)
#         return x

# class Decoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()
        
#         # init - layers
#         self.fc1 = nn.Linear(latent_dim, 16 * 14 * 14)
#         self.bn1 = nn.BatchNorm2d(num_features=16)
#         self.conv1 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1)
        
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.ConvTranspose2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the decoder        
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):
#         x = self.fc1(x)
#         x = x.view(x.size(0), 16, 14, 14) 
#         x = F.sigmoid(self.conv1(self.bn1(x)))
#         return x

#op 3

# class Encoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()

#         # init - layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
        
#         self.fc1 = nn.Linear(128 * 2 * 2, latent_dim)
  
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the encoder
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):

#         x = F.relu(self.conv1(x))

#         x = F.relu(self.conv2(x))

#         x = F.relu(self.conv3(x))

#         x = F.relu(self.conv4(x))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = self.fc1(x)
#         return x
# class Decoder(nn.Module):
#     def __init__(self, latent_dim):
#         super().__init__()
#         # init - layers
#         self.fc1 = nn.Linear(latent_dim, 128 * 2 * 2)
#         self.conv1 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
#         self.conv2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1)
#         self.conv3 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1)
#         self.conv4 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1)
        
#         # inits - weights (Xavier initialization)
#         for layer in self.modules():
#             if isinstance(layer, nn.ConvTranspose2d) or isinstance(layer, nn.Linear):
#                 init.xavier_uniform_(layer.weight)
        
#         # save total parameters of the decoder        
#         self.total_params = sum(p.numel() for p in self.parameters())


#     def forward(self, x):
#         x = self.fc1(x)
#         x = x.view(x.size(0), 128, 2, 2) 
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = F.sigmoid(self.conv4(x))
#         return x


class AutoEncoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        
        # init Model
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

        # save total parameters of the AutoEncoder 
        self.total_params = self.encoder.total_params + self.decoder.total_params

    def forward(self, x):
        encodedX = self.encoder(x)
        decodedX = self.decoder(encodedX)
        return decodedX



In [22]:
#@title Q1 
# #@title Question 1 - Auto-Encoding (also good for Q4)
autoEncoder = AutoEncoder(latent_dim).to(device)
criterion = nn.MSELoss() #measures the mean squared error (squared L2 norm) between each element in the input x and reconstructed decodedX

optimizer = optim.AdamW(autoEncoder.parameters(), lr=lr, betas=betas, weight_decay=weight_decay) #AdamW adds l2 regulization to prevent overfitting
autoEncoders = [autoEncoder]
optimizers = [optimizer]
latent_dims = [latent_dim]
print(f"Total number of parameters: {autoEncoder.total_params}")


Total number of parameters: 42699


In [ ]:
#@title # Q2 - Interpolation
# create 3 autEncoders with different latent_dim
autoEncoders = []
optimizers = []
criterion = nn.MSELoss() #measures the mean squared error (squared L2 norm) between each element in the input x and reconstructed decodedX
latent_dims = latent_dimsQ2

for i,latent_dim in enumerate((latent_dimsQ2)):
    autoEncoders.append(AutoEncoder(latent_dim).to(device))
    optimizers.append(optim.AdamW(autoEncoders[i].parameters(), lr=lr, betas=betas, weight_decay=weight_decay)) #AdamW adds l2 regulization to prevent overfitting


In [ ]:
# #@title Q3 - init encoders
# # create 3 autEncoders with different latent_dim
autoEncoders = []
optimizers = []
criterion = nn.MSELoss() #measures the mean squared error (squared L2 norm) between each element in the input x and reconstructed decodedX
latent_dims = latent_dimsQ3
for i,latent_dim in enumerate((latent_dimsQ3)):
    autoEncoders.append(AutoEncoder(latent_dim).to(device)) # init AutoEncoder
    optimizers.append(optim.AdamW(autoEncoders[i].parameters(), lr=lr, betas=betas, weight_decay=weight_decay)) #AdamW adds l2 regulization to prevent overfitting


In [8]:
#@title Test AE and calc test loss
def evaluate_loss(autoEncoder):
  
    # evaluate mode
    autoEncoder.eval()
    with torch.no_grad():
        running_loss = 0

        for data in testloader:
            # get the inputs
            images = data[0].to(device)

            # reconstruct image
            outputs = autoEncoder(images)

            # calculate and accumulate loss
            loss = criterion(outputs, images)
            running_loss += loss.item()
  

    test_loss = running_loss / len(testloader)
    return test_loss 

# Q4 - MLP,encoder Loss

def evaluate_accuracy_mlp(mlp, encoder):
    # evaluation mode
    encoder.eval()
    mlp.eval()

    # init variables
    correct_pred = 0
    total_pred = 0
    
    with torch.no_grad():
        for data in testloader:
            images, labels =  data[0].to(device), data[1].to(device)
            outputs = mlp(encoder(images))
            predictions = torch.max(outputs, 1)[1]
            # count the correct predictions
            correct_pred += (predictions == labels).sum().item()
            total_pred += labels.size(0) #count 
    
    # calculate accuracy
    accuracy = 100 * correct_pred / total_pred
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy


In [23]:
#@title Train the network & Loss function

for i in range(len(autoEncoders)):
    latent_dim = latent_dims[i]
    autoEncoder = autoEncoders[i]
    optimizer = optimizers[i]

    if (len(autoEncoders) == 1): # only 1 autoEncoder - Q1
        latent_dim = 10

    start_time = time.time()

    # use this wandb init to save autoencoder loss
    # # 🐝 1️⃣ Start a new run to track this script
    # wandb.init(
    #     # Set the project where this run will be logged
    #     project="ex2-nn", 
    #     # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    #     name=f"epochs: {epochs}, batch size: {batch_size}, lr: {lr}, filter size: {kernel_size}, latent_dim: {latent_dim}", 
    #     # Track hyperparameters and run metadata
    #     config={
    #     "learning_rate": lr,
    #     "architecture": "AutoEncoder",
    #     "dataset": "MNIST",
    #     "epochs": epochs,
    #     "latent_dim": latent_dim,
    #     })

    # Train
    for epoch in range(epochs):  # loop over the dataset multiple times
        print("##### Epoch " + str(epoch)+ " #####")               
        
        # train mode
        autoEncoder.train() 
        running_loss = 0.0

        # Train 
        for data in trainloader:
            # get the inputs
            images = data[0].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # reconstruct image
            outputs = autoEncoder(images)

            # calculate loss
            loss = criterion(outputs, images)
            
            # forward + backward + optimize
            loss.backward()
            optimizer.step()

            # accumulate loss
            running_loss += loss.item()
          

        # calculate and save train and test loss    
        train_loss = running_loss / len(trainloader)
        test_loss = evaluate_loss(autoEncoder)
        # use this wandb init to save autoencoder loss
        # wandb.log({"Train Loss": train_loss,"test Loss": test_loss})

    # calculate and print total time of training
    end_time = time.time()
    total_time = str(end_time - start_time)
    print('Finished Training, it took:' + total_time + ' seconds')

##### Epoch 0 #####
##### Epoch 1 #####
##### Epoch 2 #####
##### Epoch 3 #####
##### Epoch 4 #####
##### Epoch 5 #####
##### Epoch 6 #####
##### Epoch 7 #####
##### Epoch 8 #####
##### Epoch 9 #####
##### Epoch 10 #####
##### Epoch 11 #####
##### Epoch 12 #####
##### Epoch 13 #####
##### Epoch 14 #####
##### Epoch 15 #####
##### Epoch 16 #####
##### Epoch 17 #####
##### Epoch 18 #####
##### Epoch 19 #####
Finished Training, it took:240.2503125667572 seconds


In [ ]:
# #@title Q2 - Interpolation
# randomly choose two images from testset
idx1 = np.random.randint(low=0, high=len(testset))
idx2 = np.random.randint(low=0, high=len(testset))
img1, digit1 = testset[idx1]
img2, digit2 = testset[idx2]
alphas = [0.2, 0.4, 0.6, 0.8] # to interpolate with different alphas

# choose again till we get different digits
while(digit1 == digit2):
    idx2 = np.random.randint(0, len(testset))
    img2, digit2 = testset[idx2]

# prepare images for interploation
img1ForInter = img1.unsqueeze(dim=1).to(device)
img2ForInter = img2.unsqueeze(dim=1).to(device) 
# change dimensions - [1,28,28] -> [28,28] for displaying
img1 = img1.squeeze().cpu()
img2 = img2.squeeze().cpu()

# plot images for every autoEncoder for comperation
for i in range(len(autoEncoders)):
    print("autoEncoder with latent dim: {}".format(latent_dimsQ2[i]))
    autoEncoder = autoEncoders[i]

    # interpolate with alphas
    interImgs = []
    for alpha in alphas:
        interImg = (autoEncoder.decoder((alpha * autoEncoder.encoder(img1ForInter)) +
                                      (1-alpha) * autoEncoder.encoder(img2ForInter)))
        interImg = interImg.squeeze().cpu().detach().numpy()
        interImgs.append(interImg)

 
    # Create images & description arrays
    images = [img1,img2] + interImgs
    description = [digit1,digit2] + alphas
    
    # Create and fill subplot with images and their labels
    fig2, axs2 = plt.subplots(2, 3)
    for row in range(2):
      for col in range(3):
        descrip = "Alpha"
        if row == 0 and (col == 0 or col == 1):
            descrip = "Digit"
        axs2[row][col].imshow(images[3 * row + col], cmap='gray') # plot img1
        axs2[row][col].set_title('{}: {}'.format(descrip,description[3 * row + col]))

    plt.show()


In [ ]:
#@title Q3 - Decorrelation

# get a batch of 4000 images and labels and transfer to Device
images, labels = next(iter(trainDecorrelation)) # not going to use labels
images = images.to(device)

correlations = []

# encode images with 4 different encoders 
for i in range(len(autoEncoders)):
    latent_dim = latent_dimsQ3[i]
    print("autoEncoder with latent dim: {}".format(latent_dim))
    autoEncoder = autoEncoders[i]
    autoEncoder.encoder.eval()

    # encode images and calc perason correlation
    encodedImages = autoEncoder.encoder(images)
    corr_matrix = np.corrcoef(encodedImages.T.cpu().detach().numpy())
    #compute the correlation matrix for all pairs of coordinates and then compute an average measure of correlation
    correlation = np.mean(np.abs(corr_matrix - np.eye(corr_matrix.shape[0])))  
    correlations.append(correlation)

# Create a plot with correlationsand latent dimensions
plt.scatter(latent_dimsQ3, correlations)
# Set  x,y axis labels and title 
plt.xlabel('Latent Dimension')
plt.ylabel('Pearson Correlation')
plt.title('Pearson Correlations with respect to the latent space dimension')
plt.show()


In [24]:
#@title Q4 - Transfer Learning
    # 🐝 1️⃣ Start a new run to track this script
wandb.init(
    # Set the project where this run will be logged
    project="ex2-nn", 
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"epochs: {epochs}, batch size: {batch_size}, lr: {lr}, filter size: {kernel_size}, latent_dim: {latent_dim}", 
    # Track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "AutoEncoder",
    "dataset": "MNIST",
    "epochs": epochs,
    "latent_dim": latent_dim,
    })

#Define MLP 
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(in_features=32, out_features=64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)  
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.softmax(self.fc3(x), dim=1)
        return x


# Q4.i) train MLP only (i already have the images,labels)
def train_model1():
    # Init Models, Critetion and Optimizer        
    mlp = MLP().to(device)
    encoder = autoEncoder.encoder # pre-trained encoder
    criterionMLP = nn.CrossEntropyLoss() 
    optimizerMLP = optim.Adam(mlp.parameters(), lr=lr, betas=betas, weight_decay=weight_decay) #AdamW adds l2 regulization to prevent overfitting    
    mlp1_loss = []
    mlp1_accuracy = []


    # exclude encoder's weights from optimization
    for param in encoder.parameters():
        param.requires_grad = False

    # Train
    for epoch in range(epochs):  # loop over the dataset multiple times
        mlp.train()
        encoder.train()
        running_loss = 0
        for data in mlpTrainSet:
            images, labels = data[0].to(device),data[1].to(device)
            # zero the parameter gradients - only MLP params
            optimizerMLP.zero_grad()
            outputs = encoder(images)
            outputs = mlp(outputs)

            # calculate loss and accumulate loss
            loss = criterionMLP(outputs, labels)
            running_loss += loss.item()

             # update MLP and encoder weights
            loss.backward()
            optimizerMLP.step()

        # calculate and save train and test loss    
        mlp1_loss.append(running_loss / len(trainloader))
        mlp1_accuracy.append(evaluate_accuracy_mlp(mlp, encoder))
    
    return mlp1_loss,mlp1_accuracy
    


# Q4.ii) train MLP + Encoder        
# Init Models, Critetion and Optimizer  
def train_model2():

    mlp = MLP().to(device)
    encoder = autoEncoder.encoder # pre-trained encoder
    # include encoder's weights in optimization
    for param in encoder.parameters():
        param.requires_grad = True

    combined_params = list(mlp.parameters()) + list(encoder.parameters())
    criterionMLP = nn.CrossEntropyLoss() 
    optimizerMLP = optim.Adam(combined_params, lr=lr, betas=betas, weight_decay=weight_decay) #AdamW adds l2 regulization to prevent overfitting    
    mlp2_loss = []
    mlp2_accuracy = []
    print("second")
    # Train
    for epoch in range(epochs):  # loop over the dataset multiple times
        mlp.train()
        encoder.train()
        running_loss = 0 
        for data in mlpTrainSet:
            images, labels = data[0].to(device),data[1].to(device)
            # zero the parameter gradients - only MLP params
            optimizerMLP.zero_grad()
            outputs = encoder(images)
            outputs = mlp(outputs)

            # calculate loss and accumulate loss
            loss = criterionMLP(outputs, labels)
            running_loss += loss.item()

             # update MLP and encoder weights
            loss.backward()
            optimizerMLP.step()

        # calculate and save train and test loss    
        mlp2_loss.append(running_loss / len(trainloader))
        mlp2_accuracy.append(evaluate_accuracy_mlp(mlp, encoder))


    return mlp2_loss,mlp2_accuracy
    
mlp1_loss,mlp1_accuracy = train_model1()
mlp2_loss,mlp2_accuracy = train_model2()
for i in range(len(mlp1_loss)):
    wandb.log({"MLP1 Loss": mlp1_loss[i],"MLP2 Loss": mlp2_loss[i],"MLP1 Accuracy": mlp1_accuracy[i],"MLP2 Accuracy": mlp2_accuracy[i]})



print("done")

# calculate and save train and test loss    



MLP1 Accuracy,▁▂▃▄▅▅▆▆▇▇▇▇▇███████
MLP1 Loss,███▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▁▁
MLP2 Accuracy,▁▁▄▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
MLP2 Loss,██▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▁▁▁
MLP1 Accuracy,62.23
MLP1 Loss,0.00787
MLP2 Accuracy,60.83
MLP2 Loss,0.00747


Accuracy: 18.11%
Accuracy: 24.92%
Accuracy: 31.93%
Accuracy: 37.36%
Accuracy: 41.55%
Accuracy: 44.86%
Accuracy: 47.15%
Accuracy: 48.71%
Accuracy: 49.73%
Accuracy: 50.69%
Accuracy: 51.74%
Accuracy: 52.38%
Accuracy: 53.12%
Accuracy: 53.91%
Accuracy: 54.65%
Accuracy: 55.61%
Accuracy: 56.49%
Accuracy: 57.23%
Accuracy: 58.18%
Accuracy: 59.09%
second
Accuracy: 12.09%
Accuracy: 11.58%
Accuracy: 23.11%
Accuracy: 31.14%
Accuracy: 37.80%
Accuracy: 45.01%
Accuracy: 52.48%
Accuracy: 56.43%
Accuracy: 58.60%
Accuracy: 59.88%
Accuracy: 60.98%
Accuracy: 61.69%
Accuracy: 62.37%
Accuracy: 63.03%
Accuracy: 63.60%
Accuracy: 64.55%
Accuracy: 65.61%
Accuracy: 66.71%
Accuracy: 67.67%
Accuracy: 68.67%
done


In [ ]:
wandb.finish()